# Boolean Query

In this notebook, we will implement a Boolean information retrieval model by Elasticsearch.  
We have these steps: <br>

1- Connect to the Elasticsearch Cluster and Create an Index <br>
2- Indexing Documents <br>
3- Boolean Retrieval <br>

In [ ]:
!pip install elasticsearch

In [ ]:
from elasticsearch import Elasticsearch, helpers
import os, json, time
from datetime import datetime
from tqdm import tqdm 

## Ignore The Security Warnings
Here we ignore the security warnings. It's suggested that you don't run below cell until you ensure that all warnings are about security. 


In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Load Processed Data

In [ ]:
import json
dataset_path = 'IR_data_news_12k.json'
with open(dataset_path) as f:
    data = json.load(f)
print(data['0'].keys())

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])


## Connect to the Elasticsearch Cluster and Create an Index
After starting your Elasticsearch on your pc (localhost:9200 is the default), we have to connect to it via the following piece of code.


In [ ]:
index_name = 'my_index_name'
es = Elasticsearch("http://localhost:9200")
es.indices.create(index = index_name)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index_name'})

### Check the Cluster Status

In [ ]:
dict(es.info())

## Indexing Documents 

we can add our documents to created index in two ways:<br>


> 1- one by one in for loop. <br>
2- use [Bulk API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html#:~:text=Bulk%20APIedit,can%20greatly%20increase%20indexing%20speed.), which performs multiple indexing or delete operations in a single API call. This reduces overhead and can significantly increase indexing speed.




###  for loop by es.index 
**You dont have to run this cell**. 

In [ ]:
for i in tqdm(range(len(data))):
    es.index(index = index_name, id=i, document=data[str(i)])
    i = i + 1

100%|█████████████████████████████████████| 12202/12202 [03:41<00:00, 55.02it/s]


###  Bulk API

In [ ]:
from elasticsearch.helpers import bulk

def bulk_sync():
    actions = [
        {
            '_index': 'ta3_index',
            '_id':doc_id,
            '_source': doc
        } for doc_id,doc in data.items()
    ]
    bulk(es, actions)


In [ ]:
start = time.time()
bulk_sync()
end = time.time()
print("Indexing all documents took about {:.2f} seconds".format(end - start))

Indexing all documents took about 5.69 seconds


### check index

In [ ]:
es.count(index = index_name)

ObjectApiResponse({'count': 12202, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Boolean Retrieval 

### Bool Query Structure

It is built using one or more boolean clauses, each clause with a typed occurrence. The occurrence types are:


**must**: The clause (query) must appear in matching documents and will contribute to the score.
<br><br>
**filter**: The clause (query) must appear in matching documents. However unlike must the score of the query will be ignored. Filter clauses are executed in filter context, meaning that scoring is ignored and clauses are considered for caching.
<br><br>
**should**: The clause (query) should appear in the matching document.
<br><br>
**must_not**: The clause (query) must not appear in the matching documents. Clauses are executed in filter context meaning that scoring is ignored and clauses are considered for caching. Because scoring is ignored, a score of 0 for all documents is returned.




For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html). 


For more details, You can read documents of [Match Query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-top-level-params) and [Match Phrase Query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query-phrase.html).

In [ ]:
# sample query: تحریم هسته‌ای ً آمریکا !ایران
query= {
        "bool": {
          "should": [
              { 
                "match": {
                  "content": {
                    "query": "", # add query word 
                  }
                }
              }, 
              
              { 
                "match_phrase":{
                  "content":{
                    "query":"", # add query word 
                  }
                }  
              },
          ],
          "must_not": [
              {
                "match": {
                  "content": {
                    "query": "", # add query word 
                  }
                }
              }
          ],
        },
    }

### Search query

The _source field contains the original JSON document body that was passed at index time. The _source field itself is not indexed (and thus is not searchable), but it is stored so that it can be returned when executing fetch requests, like get or search.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html#search-api-response-body).

In [ ]:
res = es.search(index = index_name, query=query, _source= ["url"])
res = dict(res)

### Results

In [ ]:
print("{} results in {} s: ".format(res['hits']['total']['value'] ,res['took']/1000))
for doc in res['hits']['hits']:
    print(doc['_source']['url'])